In [2]:
import os
import json
import itertools
from tqdm import tqdm
from tensorflow import keras
from IPython.display import clear_output
# from collections import Counter
# import time

# import matplotlib.pyplot as plt
# import numpy as np


# from pathlib import Path





from pattern.text.en import tokenize
from gensim.models import Word2Vec

In [3]:
import argparse

parser = argparse.ArgumentParser()

parser.add_argument(
    '--synthetic_column_size',
    type=int,
    default=10,
    help='Size of synthetic column')
parser.add_argument(
    '--sequence_size',
    type=int,
    default=50,
    help='Length of word sequence of synthetic column')
parser.add_argument(
    '--model_dir',
    type=str,
    default=os.path.abspath('C:/Users/zacharias.detorakis/Desktop/nov-city-ms-project/app/w2v_model/enwiki_model'),
#     default='/w2v_model/enwiki_model/',
    help='Directory of word2vec model')
FLAGS, unparsed = parser.parse_known_args()

In [4]:
# Path(os.getcwd()+"\output\cnn_models").mkdir(parents=True, exist_ok=True)

# Load input data

* data: Tabular data + ground truth
* dict_col_candidate_classes: a dictionary with filename_columns and in each of the an array of [(candidate_type, candidate_entity, original_cell_value, rank)]
* type_neighours_pos_neg_samples: a dictionary that is used to train the classifiers so for each candidate class we have the neighbouring classes, positive samples from the KG and positive and negative samples from the tabular data

In [5]:
# Load the dictionary with the lookup results for each cell value in the tabular data
def load_json(data_json):
    with open(data_json) as json_file:
        return json.load(json_file)

In [16]:
output_folder = 'output\\'
cnn_model_directory = os.getcwd()+'\\output\\cnn_models'


data = load_json(output_folder+'data-rep.json')
# dict_col_candidate_classes = load_json(output_folder+'dict_col_candidate_classes.json')
type_neighours_pos_neg_samples = load_json(output_folder+'type_neighours_pos_neg_samples.json')
dict_cand = load_json(output_folder+'dict_cand.json')


In [17]:
def get_cnn_models(directory):
    temp = [x[0] for x in os.walk(directory)]
    temp.remove(directory)
    return set([x.replace(directory+'\\','').split('\\')[0] for x in temp])

trained_models = list(get_cnn_models(cnn_model_directory))
# trained_models

# Get predictions

In this step, provided that we have the ground truth, we asses if the expected class is in the top x of the retrieved candidate classes.

In [18]:
def load_model(cnn_model_directory, candidate_class):
    return keras.models.load_model(cnn_model_directory+'\%s' % candidate_class)

# Get avg number of words per cell value

In [19]:
def avg_cell_value_word_lenght(data):
    cell_values = list()

    for file in data:
        for col in file['data']:
            cell_values += file['data'][col]

    cell_values = list(set(cell_values))
    len(cell_values)

    word_seq = list()

    for cell_value in cell_values:
        value = str(cell_value).replace('_', ' ').replace('-', ' ').replace('.', ' ').replace('/', ' ').replace('"', ' ').replace("'", ' ')
        tokenized_line = ' '.join(tokenize(value))
        is_alpha_word_line = [word for word in tokenized_line.lower().split() if word.isalpha()]
        word_seq += is_alpha_word_line

    return len(word_seq) / len(cell_values)

avg_cell_value_word_lenght(data)

2.504999425353408

# only run once

In [10]:
w2v_model = Word2Vec.load(os.path.join(FLAGS.model_dir, 'word2vec_gensim'))

In [20]:
def generate_synthetic_columns(entities, synthetic_column_size):
    ent_units = list()
    if len(entities) >= synthetic_column_size:
        for i, ent in enumerate(entities):
            unit = list([ent])
            unit += random.sample(entities[0:i] + entities[(i + 1):], synthetic_column_size - 1)
            
            ent_units.append(unit)
    else:
        unit = entities + ['NaN'] * (len(entities) - synthetic_column_size)
        ent_units.append(unit)
    return ent_units

def synthetic_columns2sequence(ent_units, sequence_size):
    word_seq = list()
    for ent in ent_units:
        ent_n = str(ent).replace('_', ' ').replace('-', ' ').replace('.', ' ').replace('/', ' '). \
            replace('"', ' ').replace("'", ' ')
        tokenized_line = ' '.join(tokenize(ent_n))
        is_alpha_word_line = [word for word in tokenized_line.lower().split() if word.isalpha()]
        word_seq += is_alpha_word_line
    if len(word_seq) >= sequence_size:
        return word_seq[0:sequence_size]
    else:
        return word_seq + ['NaN'] * (sequence_size - len(word_seq))
    
def sequence2matrix(word_seq, sequence_size, w2v_model):
    ent_v = np.zeros((sequence_size, w2v_model.vector_size, 1))
    for i, word in enumerate(word_seq):
        if not word == 'NaN' and word in w2v_model.wv.vocab:
            w_vec = w2v_model.wv[word]
            ent_v[i] = w_vec.reshape((w2v_model.vector_size, 1))
    return ent_v

# to delete align

In [21]:
import random
import math
import numpy as np

In [22]:
def embedding(cell_values):
    synthetic_columns = generate_synthetic_columns(cell_values, FLAGS.synthetic_column_size)

    synthetic_columns_sequences = list()
    for synthetic_column in synthetic_columns:
    #     print(synthetic_column)
        synthetic_columns_sequences.append(synthetic_columns2sequence(synthetic_column, FLAGS.sequence_size))


    X = np.zeros((len(synthetic_columns_sequences), FLAGS.sequence_size, w2v_model.vector_size, 1))

    for sample_i, sequence in enumerate(synthetic_columns_sequences):
        X[sample_i] = sequence2matrix(sequence, FLAGS.sequence_size, w2v_model)

    return X

# X = embedding(unique_cell_values)

In [23]:
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow import keras
# from IPython.display import clear_output

In [30]:
limit = 5

dict_predictions = dict()

# for filename in tqdm(dict_cand):
for filename in tqdm(dict(itertools.islice(dict_cand.items(), 1))):
    dict_predictions[filename] = dict()
    for col in dict_cand[filename]:
        dict_predictions[filename][col] = dict()
        # Get the candidate classes identified in previous steps. there is a limit variable in case we want to focus on the top x portion of that list
        candidate_cls = dict_cand[filename][col]['class_without_hr'][:limit]
#         candidate_cls = trained_models
        cell_values = next(item for item in data if item["filename"] == filename)['data'][col]
        
        print(filename, col, candidate_cls)
        actual_cls = next(item for item in data if item["filename"] == filename)['gt'][col]
        dict_predictions[filename][col]['gt'] = actual_cls
        print(actual_cls)
        
        unique_cell_values=list(set(cell_values))
        X = embedding(unique_cell_values)
        
        results = list()
        for cls in candidate_cls:
            # load the model
            model = load_model(cnn_model_directory, cls)
            y_pred = tf.keras.activations.sigmoid(model.predict(X)).numpy().round()
            results.append((cls, round(y_pred.sum()*100/X.shape[0],2)))
#             print(cls)
        results = sorted(results, key=lambda x: x[1], reverse=True)
        dict_predictions[filename][col]['cand_cls'] = candidate_cls
        dict_predictions[filename][col]['pred'] = results
            
    clear_output(wait=True)   
with open(('output/dict_predictions-%s.json' % time.strftime("%Y%m%d-%H%M%S")), 'w') as fp:
        json.dump(dict_predictions, fp)        

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.38s/it]


NameError: name 'time' is not defined

In [29]:
dict_predictions['58891288_0_1117541047012405958']['1']['pred']

[('TelevisionShow', 56.0),
 ('Group', 56.0),
 ('Book', 46.0),
 ('Film', 39.0),
 ('Work', 39.0),
 ('TelevisionEpisode', 39.0),
 ('HollywoodCartoon', 38.0),
 ('Comic', 34.0),
 ('Play', 31.0),
 ('SpaceMission', 30.0),
 ('Poem', 28.0),
 ('Band', 28.0),
 ('WrittenWork', 26.0),
 ('Protein', 26.0),
 ('Manga', 26.0),
 ('Software', 23.0),
 ('VideoGame', 20.0),
 ('Horse', 19.0),
 ('WineRegion', 17.0),
 ('Brain', 16.0),
 ('Musical', 13.0),
 ('Mammal', 13.0),
 ('Artwork', 12.0),
 ('RadioProgram', 10.0),
 ('Website', 9.0),
 ('Bird', 8.0),
 ('WorldHeritageSite', 8.0),
 ('Activity', 7.0),
 ('Single', 7.0),
 ('Album', 7.0),
 ('Grape', 7.0),
 ('EthnicGroup', 6.0),
 ('TimeInterval', 6.0),
 ('NaturalPlace', 6.0),
 ('RollerCoaster', 6.0),
 ('Fashion', 6.0),
 ('Beverage', 6.0),
 ('Colour', 5.0),
 ('SupremeCourtOfTheUnitedStatesCase', 5.0),
 ('Food', 5.0),
 ('Mineral', 5.0),
 ('Settlement', 4.0),
 ('PopulatedPlace', 4.0),
 ('AnatomicalStructure', 4.0),
 ('ComedyGroup', 4.0),
 ('Currency', 4.0),
 ('Weapon', 

In [ ]:
with open(('output/dict_predictions-%s.json' % time.strftime("%Y%m%d-%H%M%S")), 'w') as fp:
        json.dump(dict_predictions, fp)

Retrieve all folders under the cnn_models root

In [ ]:
def get_cnn_models(directory):
    temp = [x[0] for x in os.walk(directory)]
    temp.remove(directory)
    return set([x.replace(directory+'\\','').split('\\')[0] for x in temp])

trained_models = list(get_cnn_models(cnn_model_directory))
trained_models

In [ ]:
# model.summary()

In [ ]:
# results